In [1]:
import visa
import numpy
import time
import array
import pyvisa

In [70]:
class AWG_Handler():

    def __init__(self):
        self.gpib_addr = 'GPIB0::4::INSTR'
        rm = pyvisa.ResourceManager()
        self.gpib = rm.open_resource(self.gpib_addr)
        self.gpib.term_chars = None
        self.gpib.timeout = 50.0


    ##########################
    ### operation commands ###
    ##########################


    def write(self, cmd):
        self.gpib.write(cmd)
        time.sleep(0.1)

    def IsOutputOn(self):
        """ returns FALSE if both outputs are switched off """
        return awg.gpib.ask(":OUTPUT1:STAT?;:OUTPUT2:STAT?") != "0;0\n"

    def On(self, channel="NORMAL"):
        #""" output channel is either "NORMAL" or "INVERTED" """
        #self.write("OUTPUT:CH1:%s:STATE ON" %channel)
        self.gpib.write(":OUTPUT1:STAT ON;:OUTPUT2:STAT ON")
        

    def Off(self, channel="NORMAL"):
        self.gpib.write(":OUTPUT1:STAT OFF;:OUTPUT2:STAT OFF")

    def ReadEventsAndClear(self):
        self.gpib.ask("*ESR?")
        self.reply = self.gpib.ask("ALLEV?")
        return self.reply

    def Reset(self):
        self.write("*RST")

    def set_data_destination(self, waveform_name):
        self.write(':DATA:DESTINATION "%s"' % waveform_name)
        self.write(':DATA:WIDTH 1') # sets number of bytes per data point for transfer

    def get_generator_source(self):
        return self.gpib.ask(":DATA:SOURCE?")

    # def set_generator_source(self, source):
        # self.write(':DATA:SOURCE "%s"' % source)

    def set_mode(self, mode):
        self.write(f'AWGC:RMOD {mode}')
        #self.write("MODE %s" % mode)

    #########################
    ### waveform commands ###
    #########################

    def upload_waveform(self, waveform, waveform_name='acube.wfm', waveform_ch2 = None, waveform_name_ch2 = None):
        self.set_data_destination(waveform_name)
        self.gpib.term_chars=''
        # transform waveform to something readable by the AWG
        #waveform *= 255.0/waveform.max() # normalize
        waveform_tosend = array.array('B',waveform)
        self.write(':CURVE #%u%u%s' % (len( len(waveform_tosend).__str__() ), len(waveform_tosend), waveform_tosend.tostring()))
        self.gpib.term_chars=None
        self.Off()
        self.set_mode('TRIG')
        self.write(':CH1:WAVEFORM "%s"' % waveform_name)
        if waveform_name_ch2 != None:
            self.set_data_destination(waveform_name_ch2)
            waveform_tosend = array.array('B',waveform_ch2)
            self.write(':CURVE #%u%u%s' % (len( len(waveform_tosend).__str__() ), len(waveform_tosend), waveform_tosend.tostring()))
            self.write(':CH2:WAVEFORM "%s"' % waveform_name_ch2)

        #write(':OUTPUT:CH1:NORM:STATE ON')

    def Power(self, power = None):
        if power == None:
            amp = float(self.gpib.ask(':CH1:AMPLITUDE?'))
            return 10*numpy.log10(amp**2 / 50/2 / 1e-3)
        else:
            newamp = numpy.sqrt(.1*10**(power/10))
            self.write(':CH1:AMPLITUDE %3.2fV' % newamp)

    def Amplitude(self, amp= None):
        if amp == None:
            amp = float(self.gpib.ask(':CH1:AMPLITUDE?'))
            return amp
        else:
            self.write(':CH1:AMPLITUDE %3.2fV' % amp)

    def del_waveform(self, waveform_name):
        """ Deletes the specified waveform. Passing "ALL" deletes all waveforms."""
        if waveform_name == 'ALL':
            self.write("WLIST:WAVEFORM:DELETE ALL")
        else:
            self.write("WLIST:WAVEFORM:DELETE %s" % waveform_name)


    def sine_wave(self, frequency, duration, phase=0, step = 1):
        waveform = .5*(1+numpy.sin(numpy.arange(0, duration, step)*2*numpy.pi*frequency*1e-9 + phase))
        return numpy.array([int(a) for a in (0xFF * waveform)])

    def generate_pulse_waveform(self, pulse_sequence, atom = 640):
        waveform = numpy.array([], dtype=int)
        duration = 0
        for element in pulse_sequence:
            waveform = numpy.append(waveform, self.sine_wave(element[0], element[1]))
            duration += element[1]
        if duration % atom != 0:
            waveform = numpy.append(waveform, numpy.zeros((atom - duration % atom), dtype = numpy.int))
            # print waveform
        return waveform, duration

In [71]:
awg = AWG_Handler()

In [18]:
awg.upload_waveform(256*[int(0xFF*0.5)], waveform_name='pleasework.wfm')

In [20]:
awg.Amplitude(1)

In [38]:
awg.set_mode('CONT')

In [39]:
awg.IsOutputOn()

Traceback (most recent call last):
  File "C:\qudi\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-39-a09497455848>", line 1, in <module>
    awg.IsOutputOn()
  File "<ipython-input-36-b80fd3ac1396>", line 22, in IsOutputOn
    return self.gpib.ask(":OUTPUT:CH1?") != "0;0"
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 569, in ask
    return self.query(message, delay)
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 564, in query
    return self.read()
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 413, in read
    message = self._read_raw().decode(enco)
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 386, in _read_raw
    chunk, status = self.visalib.read(self.session, size)
  File "C:\Users\P

In [65]:
awg.gpib.ask(":OUTPUT1:STAT?;:OUTPUT2:STAT?") != 

'0;0\n'

In [51]:
awg.gpib.write('AWGC:RUN')

(10, <StatusCode.success: 0>)

In [55]:
awg.gpib.write(":OUTPUT1:STAT OFF;:OUTPUT2:STAT OFF")

(37, <StatusCode.success: 0>)

In [76]:
awg.IsOutputOn()

False

In [84]:
awg.gpib.ask(":DATA:DESTINATION?")

Traceback (most recent call last):
  File "C:\qudi\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-84-4a037107b08c>", line 1, in <module>
    awg.gpib.ask(":DATA:DESTINATION?")
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 569, in ask
    return self.query(message, delay)
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 564, in query
    return self.read()
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 413, in read
    message = self._read_raw().decode(enco)
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 386, in _read_raw
    chunk, status = self.visalib.read(self.session, size)
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\ctwrapper\functions.py", line 1584, in read
    r

In [87]:
awg.gpib.ask(':DATA:WIDTH?')

Traceback (most recent call last):
  File "C:\qudi\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-87-aaaea12cb430>", line 1, in <module>
    awg.gpib.ask(':DATA:WIDTH?')
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 569, in ask
    return self.query(message, delay)
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 564, in query
    return self.read()
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 413, in read
    message = self._read_raw().decode(enco)
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 386, in _read_raw
    chunk, status = self.visalib.read(self.session, size)
  File "C:\Users\PI3\anaconda3\envs\qudi\lib\site-packages\pyvisa\ctwrapper\functions.py", line 1584, in read
    ret = l

Traceback (most recent call last):
  File "C:\qudi\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-109-5824847dd5bd>", line 1, in <module>
    awg.get_file_data("test3.wfm")
AttributeError: 'AWG_Handler' object has no attribute 'get_file_data'
Traceback (most recent call last):
  File "C:\qudi\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-110-3a71ee2754b7>", line 1, in <module>
    file = awg.get_file_data("test3.wfm")
AttributeError: 'AWG_Handler' object has no attribute 'get_file_data'
Traceback (most recent call last):
  File "C:\qudi\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-111-be0118a838dc>", line 1, in <module>
    file = awg.gpib.get_file_data("test3.wfm")
AttributeError: 'GPIBInstrument' object has

Traceback (most recent call last):
  File "C:\qudi\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-113-3a71ee2754b7>", line 1, in <module>
    file = awg.get_file_data("test3.wfm")
AttributeError: 'AWG_Handler' object has no attribute 'get_file_data'
Traceback (most recent call last):
  File "C:\qudi\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-114-bbe02f946d54>", line 1, in <module>
    file
NameError: name 'file' is not defined
Traceback (most recent call last):
  File "C:\qudi\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-115-d2eb759d13d7>", line 1, in <module>
    awg.reset()
AttributeError: 'AWG_Handler' object has no attribute 'reset'
Traceback (most recent call last):
  File "C:\qudi\qudi\logic\jupyterk

In [82]:
waveform_name = "please.wfm"
awg.write(f':DATA:DESTINATION "{waveform_name}"')